In [1]:
from anthropic import Anthropic
import os

anthropic = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

In [39]:
from IPython.display import display_javascript, display_html, Javascript, HTML
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
# display_javascript(Javascript("""
# <div>
# <link rel="stylesheet"
#       href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/10.0.3/styles/default.min.css">
# <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/10.0.3/highlight.min.js"></script>
# <script>hljs.initHighlightingOnLoad();</script>
# </div>"""))


code = """def fib(n):
    a, b = 0, 1
    while True:
        print(a, end=' ')
        a, b = b, a+b
        print()
fib(1000)
"""

# highlighted_code = highlight(code, PythonLexer(), HtmlFormatter())

# display_html(HTML("""
# <div>
# <pre><code class="python"></code></pre>
# </div>
# """))

formatter = HtmlFormatter(style="default", linenos="inline")
# formatter.nobackground = False
# css = formatter.get_style_defs('.highlight')
display(HTML(f'<style>{formatter.get_style_defs(".highlight")}</style>'))
display(HTML(highlight(code, PythonLexer(), formatter)))


In [42]:
print(formatter.get_style_defs(".highlight"))
display(HTML(f'<style>{formatter.get_style_defs(".highlight")}</style>'))

pre { line-height: 125%; }
td.linenos .normal { color: inherit; background-color: transparent; padding-left: 5px; padding-right: 5px; }
span.linenos { color: inherit; background-color: transparent; padding-left: 5px; padding-right: 5px; }
td.linenos .special { color: #000000; background-color: #ffffc0; padding-left: 5px; padding-right: 5px; }
span.linenos.special { color: #000000; background-color: #ffffc0; padding-left: 5px; padding-right: 5px; }
.highlight .hll { background-color: #ffffcc }
.highlight { background: #f8f8f8; }
.highlight .c { color: #3D7B7B; font-style: italic } /* Comment */
.highlight .err { border: 1px solid #F00 } /* Error */
.highlight .k { color: #008000; font-weight: bold } /* Keyword */
.highlight .o { color: #666 } /* Operator */
.highlight .ch { color: #3D7B7B; font-style: italic } /* Comment.Hashbang */
.highlight .cm { color: #3D7B7B; font-style: italic } /* Comment.Multiline */
.highlight .cp { color: #9C6500 } /* Comment.Preproc */
.highlight .cpf { colo

In [4]:

prompt_template = """You are given the text of a function that will be run on private data. You are tasked with evaluating whether the function is safe to run and whether the function is leaking private information. The private data will be passed as the arguments of the function. Your decision will be used on real data.

Here is the function:

<code>
{code}
</code>

After analyzing the code, make a decision on whether it is suitable to run it, or if it is a security risk or a privacy leak.

Provide a brief explanation for your decisions, highlighting the key factors that influenced your choice.
Also explain the main parts of the code while highlighting which lines of the code are you referring to.
Please specify which lines you are referring to at the start of the line. 

Format your response as follows:

<evaluation>
<decision>[SUITABLE/SECURITY_RISK/PRIVACY_LEAK]</decision>
<explanation>
[Your brief explanation here]
</explanation>
</evaluation>
"""
prompt = prompt_template.format(code=code)

message = anthropic.messages.create(
    model="claude-3-5-sonnet-latest",
    max_tokens=1024,
    messages=[{
        "role": "user",
        "content": prompt
    }]
)


In [5]:
print(message.content[0].text)

<evaluation>
<decision>SUITABLE</decision>
<explanation>
The code is a basic Fibonacci sequence generator and does not pose any security or privacy risks:

Line 1: `def fib(n):` - Defines a function that takes a single numerical parameter
Lines 2-5: The function implements a simple Fibonacci sequence calculator:
- Line 2: Initializes two variables with starting Fibonacci values
- Line 3: Creates an infinite loop
- Line 4: Prints current number with space
- Line 5: Calculates next Fibonacci numbers
- Line 6: Prints newline
Line 7: Calls the function with n=1000

Security analysis:
1. No access to system resources
2. No file operations
3. No network connections
4. No data storage
5. Input parameter 'n' is not used in calculations (though this is a bug)
6. Only outputs calculated Fibonacci numbers

Note: While the code is safe from a security perspective, it has a logical error - it creates an infinite loop and doesn't use the input parameter 'n'. This could cause resource usage issues bu

In [6]:
import re

line_dict = {}

for i, line in enumerate(message.content[0].text.split('\n')):
    match = re.match(r'.*(Line|Lines) ([0-9]*-[0-9]*|[0-9]*)', line)
    if match:
        line_dict[i] = []
        print(line) 
        for text in match.group(0).split(' ')[-1].split('-'):
            line_dict[i].append(int(text))

Line 1: `def fib(n):` - Defines a function that takes a single numerical parameter
Lines 2-5: The function implements a simple Fibonacci sequence calculator:
- Line 2: Initializes two variables with starting Fibonacci values
- Line 3: Creates an infinite loop
- Line 4: Prints current number with space
- Line 5: Calculates next Fibonacci numbers
- Line 6: Prints newline
Line 7: Calls the function with n=1000


In [7]:
line_dict

{5: [1], 6: [2, 5], 7: [2], 8: [3], 9: [4], 10: [5], 11: [6], 12: [7]}

In [8]:
from bs4 import BeautifulSoup
import re

soup = BeautifulSoup(message.content[0].text, 'html.parser')

soup.select('explanation')[0].get_text()

"\nThe code is a basic Fibonacci sequence generator and does not pose any security or privacy risks:\n\nLine 1: `def fib(n):` - Defines a function that takes a single numerical parameter\nLines 2-5: The function implements a simple Fibonacci sequence calculator:\n- Line 2: Initializes two variables with starting Fibonacci values\n- Line 3: Creates an infinite loop\n- Line 4: Prints current number with space\n- Line 5: Calculates next Fibonacci numbers\n- Line 6: Prints newline\nLine 7: Calls the function with n=1000\n\nSecurity analysis:\n1. No access to system resources\n2. No file operations\n3. No network connections\n4. No data storage\n5. Input parameter 'n' is not used in calculations (though this is a bug)\n6. Only outputs calculated Fibonacci numbers\n\nNote: While the code is safe from a security perspective, it has a logical error - it creates an infinite loop and doesn't use the input parameter 'n'. This could cause resource usage issues but not security problems.\n\nThe fun

In [9]:
HTML(highlight(code, PythonLexer(), formatter))

In [10]:

html_code = f"""
<div style="display: flex; justify-content: space-between; width: 100%;">
    <div style="width: 45%; padding: 10px; text-align: left; white-space: pre-line; display: flex;">
        LLM Agent Evaluation: {soup.select('decision')[0].get_text()}
        
        LLM Agent Explanation:
        {soup.select('explanation')[0].get_text()}
    </div>
    <div style="width: 45%; padding: 10px;  display: flex;">
       {highlight(code, PythonLexer(), formatter)}
    </div>
</div>
"""

display(HTML(html_code))

In [11]:
html_code = """
<style>
    .container {
        display: flex;
        justify-content: space-between;
        width: 100%;
    }
    .side {
        width: 45%;
        padding: 10px;
        border: 1px solid black;
    }
    .highlight {
        background-color: green;
    }
</style>

<div class="container">
    <div class="side" id="left">
        <div data-link="1">Left Side Text 1</div>
        <div data-link="2">Left Side Text 2</div>
        <div data-link="3">Left Side Text 3</div>
    </div>
    <div class="side" id="right">
        <div data-link="2">Right Side Text 1</div>
        <div data-link="3">Right Side Text 2</div>
        <div data-link="1">Right Side Text 3</div>
    </div>
</div>

<script>
    document.querySelectorAll("[data-link]").forEach(element => {
        element.addEventListener("mouseover", function() {
            let linkId = this.getAttribute("data-link");
            document.querySelectorAll('[data-link="' + linkId + '"]').forEach(el => el.classList.add("highlight"));
        });

        element.addEventListener("mouseout", function() {
            let linkId = this.getAttribute("data-link");
            document.querySelectorAll('[data-link="' + linkId + '"]').forEach(el => el.classList.remove("highlight"));
        });
    });
</script>
"""

display(HTML(html_code))

In [18]:
line_dict

{5: [1], 6: [2], 7: [3, 6], 11: [7]}

In [12]:
llm_explanation = soup.select('explanation')[0].get_text()
print(llm_explanation)


The code is a basic Fibonacci sequence generator and does not pose any security or privacy risks:

Line 1: `def fib(n):` - Defines a function that takes a single numerical parameter
Lines 2-5: The function implements a simple Fibonacci sequence calculator:
- Line 2: Initializes two variables with starting Fibonacci values
- Line 3: Creates an infinite loop
- Line 4: Prints current number with space
- Line 5: Calculates next Fibonacci numbers
- Line 6: Prints newline
Line 7: Calls the function with n=1000

Security analysis:
1. No access to system resources
2. No file operations
3. No network connections
4. No data storage
5. Input parameter 'n' is not used in calculations (though this is a bug)
6. Only outputs calculated Fibonacci numbers

Note: While the code is safe from a security perspective, it has a logical error - it creates an infinite loop and doesn't use the input parameter 'n'. This could cause resource usage issues but not security problems.

The function only outputs math

In [59]:
new_llm_explanation_lines = []
for i, line in enumerate(llm_explanation.split('\n')):
    if (i+2) in line_dict:
        if len(line_dict[i+2]) == 1:
            numbers_str = str(line_dict[i+2][0])
        else:
            numbers_str = " ".join([str(x) for x in range(line_dict[i+2][0], line_dict[i+2][1] + 1)])
        
        new_line = f"""<span data-link="{numbers_str}">{line}</span>"""
        new_llm_explanation_lines.append(new_line)
    else:
        new_llm_explanation_lines.append(line)

In [60]:
new_llm_explanation_nounhighlight = "\n".join(new_llm_explanation_lines)

In [57]:
print(new_llm_explanation)


The code is a basic Fibonacci sequence generator and does not pose any security or privacy risks:

<span class="syft-unhighlight" data-link="1">Line 1: `def fib(n):` - Defines a function that takes a single numerical parameter</span>
<span class="syft-unhighlight" data-link="2 3 4 5">Lines 2-5: The function implements a simple Fibonacci sequence calculator:</span>
<span class="syft-unhighlight" data-link="2">- Line 2: Initializes two variables with starting Fibonacci values</span>
<span class="syft-unhighlight" data-link="3">- Line 3: Creates an infinite loop</span>
<span class="syft-unhighlight" data-link="4">- Line 4: Prints current number with space</span>
<span class="syft-unhighlight" data-link="5">- Line 5: Calculates next Fibonacci numbers</span>
<span class="syft-unhighlight" data-link="6">- Line 6: Prints newline</span>
<span class="syft-unhighlight" data-link="7">Line 7: Calls the function with n=1000</span>

Security analysis:
1. No access to system resources
2. No file ope

In [16]:
line_dict

{5: [1], 6: [2, 5], 7: [2], 8: [3], 9: [4], 10: [5], 11: [6], 12: [7]}

In [58]:
# <p data-link="1">Left Side Text A (Group 1)</p>
# <p data-link="1 2">Left Side Text B (Group 1 & 2)</p>
# <p data-link="2 3">Left Side Text C (Group 2 & 3)</p>
# <p data-link="3">Left Side Text D (Group 3)</p>

css_code = """<style>
    .container {
        display: flex;
        justify-content: space-between;
        width: 100%;
    }
    .side {
        width: 45%;
        border: 1px solid black;
        white-space: pre-line;
    }
    .syft-highlight {
        background-color: yellow;
    }
    .syft-unhighlight {
        background-color: #D3D3D3;
    }
</style>
"""

html_code = f"""
{css_code}
<div class="container">
    <div class="side" id="left">
        
        LLM Agent Evaluation: {soup.select('decision')[0].get_text()}
        
        LLM Agent Explanation:
        {new_llm_explanation}
    </div>
    <div class="side" id="right">
        <div class="highlight">
            <pre>
                <span></span>
                <span class="syft-unhighlight" data-link="1"><span class="linenos">1</span><span class="k">def</span><span class="w"> </span><span class="nf">fib</span><span class="p">(</span><span class="n">n</span><span class="p">):</span></span>
                <span class="syft-unhighlight" data-link="2"><span class="linenos">2</span>    <span class="n">a</span><span class="p">,</span> <span class="n">b</span> <span class="o">=</span> <span class="mi">0</span><span class="p">,</span> <span class="mi">1</span></span>
                <span class="syft-unhighlight" data-link="3 4 5 6"><span class="linenos">3</span>    <span class="k">while</span> <span class="n">a</span> <span class="o">&lt;</span> <span class="n">n</span><span class="p">:</span></span>
                <span class="syft-unhighlight" data-link="3 4 5 6"><span class="linenos">4</span>        <span class="nb">print</span><span class="p">(</span><span class="n">a</span><span class="p">,</span> <span class="n">end</span><span class="o">=</span><span class="s1">&#39; &#39;</span><span class="p">)</span></span>
                <span class="syft-unhighlight" data-link="3 4 5 6"><span class="linenos">5</span>        <span class="n">a</span><span class="p">,</span> <span class="n">b</span> <span class="o">=</span> <span class="n">b</span><span class="p">,</span> <span class="n">a</span><span class="o">+</span><span class="n">b</span></span>
                <span class="syft-unhighlight" data-link="3 4 5 6"><span class="linenos">6</span>        <span class="nb">print</span><span class="p">()</span></span>
                <span class="syft-unhighlight" data-link="7"><span class="linenos">7</span><span class="n">fib</span><span class="p">(</span><span class="mi">1000</span><span class="p">)</span></span>
            </pre>
        </div>
    </div>
</div>
"""+"""
<script>
    document.querySelectorAll("[data-link]").forEach(element => {
        element.addEventListener("mouseover", function() {
            let linkGroups = this.getAttribute("data-link").split(" "); // Get all linked groups
            linkGroups.forEach(linkId => {
                document.querySelectorAll('[data-link~="' + linkId + '"]').forEach(el => {
                    el.classList.add("syft-highlight")
                    el.classList.remove("syft-unhighlight")
                });
            });
        });

        element.addEventListener("mouseout", function() {
            let linkGroups = this.getAttribute("data-link").split(" ");
            linkGroups.forEach(linkId => {
                document.querySelectorAll('[data-link~="' + linkId + '"]').forEach(el => {
                    el.classList.remove("syft-highlight")
                    el.classList.add("syft-unhighlight")
                });
            });
        });
    });
</script>
"""

display(HTML(f'<style>{formatter.get_style_defs(".highlight")}{css_code}</style>'))
display(HTML(html_code))

In [68]:
# <p data-link="1">Left Side Text A (Group 1)</p>
# <p data-link="1 2">Left Side Text B (Group 1 & 2)</p>
# <p data-link="2 3">Left Side Text C (Group 2 & 3)</p>
# <p data-link="3">Left Side Text D (Group 3)</p>

css_code = """<style>
    .container {
        display: flex;
        justify-content: space-between;
        width: 100%;
    }
    .side {
        width: 45%;
        border: 1px solid black;
        white-space: pre-line;
    }
    .syft-highlight {
        background-color: yellow;
    }
    .syft-unhighlight {
        background-color: #D3D3D3;
    }
</style>
"""

html_code = f"""
{css_code}
<div class="container">
    <div class="side" id="left">
        
        LLM Agent Evaluation: {soup.select('decision')[0].get_text()}
        
        LLM Agent Explanation:
        {new_llm_explanation_nounhighlight}
    </div>
    <div class="side" id="right">
        <div class="highlight">
            <pre>
<span></span>
<span data-link="1"><span class="linenos">1</span><span class="k">def</span><span class="w"> </span><span class="nf">fib</span><span class="p">(</span><span class="n">n</span><span class="p">):</span></span>
<span data-link="2"><span class="linenos">2</span>    <span class="n">a</span><span class="p">,</span> <span class="n">b</span> <span class="o">=</span> <span class="mi">0</span><span class="p">,</span> <span class="mi">1</span></span>
<span data-link="3 4 5 6"><span class="linenos">3</span>    <span class="k">while</span> <span class="n">a</span> <span class="o">&lt;</span> <span class="n">n</span><span class="p">:</span></span>
<span data-link="3 4 5 6"><span class="linenos">4</span>        <span class="nb">print</span><span class="p">(</span><span class="n">a</span><span class="p">,</span> <span class="n">end</span><span class="o">=</span><span class="s1">&#39; &#39;</span><span class="p">)</span></span>
<span data-link="3 4 5 6"><span class="linenos">5</span>        <span class="n">a</span><span class="p">,</span> <span class="n">b</span> <span class="o">=</span> <span class="n">b</span><span class="p">,</span> <span class="n">a</span><span class="o">+</span><span class="n">b</span></span>
<span data-link="3 4 5 6"><span class="linenos">6</span>        <span class="nb">print</span><span class="p">()</span></span>
<span data-link="7"><span class="linenos">7</span><span class="n">fib</span><span class="p">(</span><span class="mi">1000</span><span class="p">)</span></span>
            </pre>
        </div>
    </div>
</div>
"""+"""
<script>
    document.querySelectorAll("[data-link]").forEach(element => {
        element.addEventListener("mouseover", function() {
            let linkGroups = this.getAttribute("data-link").split(" "); // Get all linked groups
            linkGroups.forEach(linkId => {
                document.querySelectorAll('[data-link~="' + linkId + '"]').forEach(el => {
                    el.classList.add("syft-highlight")
                });
            });
        });

        element.addEventListener("mouseout", function() {
            let linkGroups = this.getAttribute("data-link").split(" ");
            linkGroups.forEach(linkId => {
                document.querySelectorAll('[data-link~="' + linkId + '"]').forEach(el => {
                    el.classList.remove("syft-highlight")
                });
            });
        });
    });
</script>
"""

display(HTML(f'<style>{formatter.get_style_defs(".highlight")}{css_code}</style>'))
display(HTML(html_code))

In [36]:
from IPython.display import Javascript, display

js_code = """
var styles = window.getComputedStyle(document.documentElement);
var allStyles = [];
for (var i = 0; i < styles.length; i++) {
    allStyles.push(styles[i] + ": " + styles.getPropertyValue(styles[i]));
}
var output = allStyles.join("\\n");

// Create a new output cell in the notebook
var cell = document.createElement("div");
cell.style.whiteSpace = "pre-wrap";  // Preserve formatting
cell.textContent = output;

// Append to the last output cell in Jupyter
document.querySelector(".output_area").appendChild(cell);
"""

display(Javascript(js_code))


<IPython.core.display.Javascript object>